In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

x_cols = ['crossing', 'finishing', 'heading_accuracy','short_passing', 'volleys', 'dribbling', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_positioning','gk_reflexes']

y_cols = ['Result']

full_data = pd.read_csv('Full_Data_feats.csv' )

print(full_data[x_cols].describe())

#Finding misclassifications

def misclassification_report(df, y_pred, y_test ):
    
    misclassified = pd.DataFrame()
    
    for count,value  in enumerate(y_test.iterrows()):
        
        if y_pred[count]!=int(value[1]['Result']):
            new_row = df.iloc[value[0]]
            new_row['Predicted'] = y_pred[count]
            misclassified = misclassified.append(new_row)
    return misclassified


In [88]:
x_all = full_data[x_cols[:]]
y_all = full_data[['Result']]


In [89]:
# from sklearn.decomposition import PCA
# comps = 10
# pca = PCA(n_components=comps, svd_solver='full')
# x_pca = pca.fit_transform(x_all)  
# print(x_pca.shape)

In [90]:
x_train,x_test,y_train,y_test = train_test_split(x_pca, y_all, test_size = 0.25, random_state = 42)


In [91]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

def create_model(x_train,y_train):
    """ Creates a Sequential model made out of Densely connected Neural Network layers. All parameters are configured as per specification for Q4"""

    model = Sequential()
    #Input Layer with 61 inputs corresponding to 61 features 
    model.add(Dense(20,input_dim = comps, activation = 'relu'))

    model.add(Dense(100,activation = 'relu'))
    #model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'sigmoid'))
  
    #Compiling the model with appropriate parameters
    model.compile(loss = 'mse', optimizer ='adam',metrics = ['accuracy'])

    result = model.fit(x_train,y_train,epochs = 50, batch_size = 50)

    #Store the model, its average training accuracy and validation accuracy over 10 epochs

    return model

#Training neural network
print(len(x_train))
model = create_model(x_train,y_train)
#pred_y = model.predict(x_test)
print(model.metrics_names)
score = model.evaluate(x_test,y_test)
print(score)

285
Epoch 1/50
285/285 [==============================] - 1s 3ms/step - loss: 0.7518 - acc: 0.4316
Epoch 2/50
285/285 [==============================] - 0s 70us/step - loss: 0.6920 - acc: 0.3509
Epoch 3/50
285/285 [==============================] - 0s 70us/step - loss: 0.6767 - acc: 0.3333
Epoch 4/50
285/285 [==============================] - 0s 70us/step - loss: 0.6528 - acc: 0.3684
Epoch 5/50
285/285 [==============================] - 0s 70us/step - loss: 0.6363 - acc: 0.3719
Epoch 6/50
285/285 [==============================] - 0s 56us/step - loss: 0.6212 - acc: 0.3614
Epoch 7/50
285/285 [==============================] - 0s 56us/step - loss: 0.6131 - acc: 0.3649
Epoch 8/50
285/285 [==============================] - 0s 56us/step - loss: 0.6080 - acc: 0.3965
Epoch 9/50
285/285 [==============================] - 0s 42us/step - loss: 0.5994 - acc: 0.4035
Epoch 10/50
285/285 [==============================] - 0s 56us/step - loss: 0.5939 - acc: 0.4000
Epoch 11/50
285/285 [===============

In [9]:
# #Training linear regression on the goal difference 
# from sklearn.linear_model import LinearRegression
# import math
# lr = LinearRegression()
# lr.fit(x_train,y_train['Goal_Diff'])
# y_pred = lr.predict(x_test[x_cols])
# goals = [math.ceil(value) for value in y_pred]

# pred_res = [1 if value > 0 else (-1 if value < 0 else 0) for value in goals]
# z = accuracy_score(y_test['Result'],pred_res)
# print(z)

In [92]:
#Train SVM
from sklearn.svm import LinearSVC, SVC

svm = SVC(C = 0.1, kernel = 'poly')
svm.fit(x_train,y_train)
# print(svm.coef_)
# print(svm.intercept_)
y_pred = svm.predict(x_test)
z =accuracy_score(y_test,y_pred)
print(z)
print(y_pred)
#mc = misclassification_report(full_data,y_pred,y_test)
#print(mc[x_cols + y_cols + ['Predicted']])


D:\Apps\Anaconda3\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Apps\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.3894736842105263
[ 1 -1  1  1  1  1  1 -1 -1 -1 -1  1  1  0  1 -1  1  1  1  1 -1 -1 -1  1
  1  1  0  1 -1  0 -1 -1  1  1 -1  1  1 -1  1 -1 -1 -1  1  1 -1 -1 -1 -1
 -1  1  1  1  0 -1  1 -1  1  1  1 -1  1  0  1 -1  0 -1  0  0  1  1 -1  1
  1  1  1  0  0 -1  1 -1 -1  1 -1  1  1 -1 -1 -1 -1  1  0  1 -1  1  1]


In [84]:
#Training decision tree
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

dt = DecisionTreeClassifier(criterion = 'entropy', max_depth = 222, random_state = 5)

dt.fit(x_train,y_train)

y_pred = dt.predict(x_test)

#y_test_synth = [-1 if total < 0 else (1 if total > 0 else 0) for total in x_test['Total'].values] 

a = accuracy_score(y_test,y_pred)
print(a)
# print(te_y.values)
#mc = misclassification_report(full_data, y_pred,y_test)
#print(mc)

0.38095238095238093
